In [ ]:
#step2: EYFP quantification for tiff file with cellpose

In [1]:
import tifffile as tiff
from cellpose import models
import numpy as np
from cellpose import models, plot
import matplotlib.pyplot as plt
from skimage import io, measure
from skimage.measure import label
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, measure
from skimage.segmentation import find_boundaries
from skimage.measure import regionprops, regionprops_table
from skimage.util import img_as_ubyte
import pandas as pd
import glob


plot_dir = '/Volumes/lsa-kkoutmou/migratedData/Private_Folders/mlruan/First_paper/data_R_script/Fig3_imaging_IF/imaging_quantification_all_pics_excels/cellpose/plot'
csv_dir = '/Volumes/lsa-kkoutmou/migratedData/Private_Folders/mlruan/First_paper/data_R_script/Fig3_imaging_IF/imaging_quantification_all_pics_excels/cellpose/CSVs'

# Ensure directories exist
os.makedirs(plot_dir, exist_ok=True)
#os.makedirs(mask_dir, exist_ok=True)
os.makedirs(csv_dir, exist_ok=True)

desired_diameter = 7  # Diameter for nuclei segmentation
desired_diameter2 = 20  # Diameter for cytoplasm segmentation

def load_and_segment_image(path, desired_diameter, flow_threshold,cellprob_threshold,niter_value=500, desired_diameter2=20, n=2):
    """
    Load a TIFF image, display channels, run Cellpose to segment nuclei,
    and overlay the segmentation contours.

    Parameters:
        path (str): Path to the TIFF file.

    Returns:
        image (numpy.ndarray): The loaded 2D image used for segmentation.
    """
    # Load the image
    image = io.imread(path)
    basename = os.path.splitext(os.path.basename(path))[0]
    mask_base = '/Volumes/lsa-kkoutmou/migratedData/Private_Folders/mlruan/First_paper/data_R_script/Fig3_imaging_IF/imaging_quantification_all_pics_excels/cellpose/mask'
    mask_dir = os.path.join(mask_base, basename)
    os.makedirs(mask_dir, exist_ok=True)

    print(f"Image shape (for channel display): {image.shape}")

    ##################################################################
    # Step 1: Display each channel
    if image.ndim == 3:
        fig, axes = plt.subplots(1, image.shape[2], figsize=(5 * image.shape[2], 5))
        if image.shape[2] == 1:
            axes = [axes]  # If only one channel, make axes a list
        for i in range(image.shape[2]):
            axes[i].imshow(image[:, :, i], cmap='gray')
            axes[i].set_title(f'Channel {i}')
            axes[i].axis('off')
        plt.tight_layout()
        plt.show()
    else:
        # If image is 2D (single channel), just show it
        plt.imshow(image, cmap='gray')
        plt.title('Single Channel')
        plt.axis('off')
        plt.show()


    
    ##################################################################
    # Step 2: Nucleus mask gaining - Run Cellpose model
   
    if image.ndim == 3:
        # Extract the first channel for nuclei segmentation
        image_for_segmentation = image[:, :, 0]
    else:
        print("The image does not have 3 channels.")

    model = models.Cellpose(gpu=False, model_type='nuclei')
    nucleus_mask, flows, styles, diams = model.eval(
        image_for_segmentation,
        diameter=desired_diameter,  # Increase if strong dots are larger
        channels=[0, 0],
        flow_threshold=flow_threshold,
        cellprob_threshold=cellprob_threshold,
        niter=niter_value)

    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(5, 5))

    # Display the fluorescence image
    ax.imshow(image_for_segmentation, cmap='gray')

    # Find contours at a constant value of 0.5
    contours = measure.find_contours(nucleus_mask, level=0.5)

    # Overlay each contour
    for contour in contours:
        ax.plot(contour[:, 1], contour[:, 0], linewidth=1, color='red')

    ax.axis('off')
    plt.title('Fluorescence Image with Nuclei Mask Edges')
    nucleus_mask_path = os.path.join(mask_dir, f"{basename}_nucleus_mask.png")
    io.imsave(nucleus_mask_path, nucleus_mask.astype(np.uint16))
    plt.show()
    # Save nucleus mask



    ##################################################################
    # Step 3: Cytoplasm mask gaining - Cytoplasm segmentation (slice 2)

    image_cytoplasm = image[ :, :,n]
#    elif image.ndim == 4:
#       image_cytoplasm = image[:, :,3]
    if image_cytoplasm is not None:
        model_cyto = models.Cellpose(gpu=False, model_type='cyto')
        cytoplasm_mask, flows, styles, diams = model_cyto.eval(
            image_cytoplasm, diameter=desired_diameter2, channels=[0, 0]
        )
    
        fig, ax = plt.subplots(figsize=(5, 5))
        ax.imshow(image_cytoplasm, cmap='gray')
    
        contours = measure.find_contours(cytoplasm_mask, level=0.5)
        for contour in contours:
            ax.plot(contour[:, 1], contour[:, 0], linewidth=1, color='blue')
    
        ax.axis('off')
        plt.title('Fluorescence Image with Cytoplasm Mask Edges')
    
        cyto_mask_path = os.path.join(mask_dir, f"{basename}_cytoplasm_mask.png")
        io.imsave(cyto_mask_path, cytoplasm_mask.astype(np.uint16))
    
        plt.show()


    ##################################################################
    # Step 4: Assign cytoplasm labels to nuclei
    if cytoplasm_mask is not None:
        nucleus_labels = label(nucleus_mask)  # Use the skimage label function
    print(f"Number of nuclei detected: {np.max(nucleus_labels)}")

    new_nucleus_labels = np.zeros_like(nucleus_labels)

    for region in measure.regionprops(nucleus_labels):
        coords = region.coords
        cytoplasm_labels = cytoplasm_mask[coords[:, 0], coords[:, 1]]
        unique_labels, counts = np.unique(cytoplasm_labels[cytoplasm_labels > 0], return_counts=True)
        if len(unique_labels) > 0:
            assigned_label = unique_labels[np.argmax(counts)]
            new_nucleus_labels[coords[:, 0], coords[:, 1]] = assigned_label

    # Save the relabeled nucleus mask
    new_nucleus_path = os.path.join(mask_dir, f"{basename}_new_nucleus_labels.png")
    io.imsave(new_nucleus_path, new_nucleus_labels.astype(np.uint16))

    ##################################################################
    # Step 5: Visualize cytoplasm and relabeled nuclei side by side
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    masks = [cytoplasm_mask, new_nucleus_labels]
    titles = ['Cytoplasm Mask', 'Nucleus Mask (After Relabeling)']

    for ax, mask, title in zip(axes, masks, titles):
        ax.imshow(mask, cmap='nipy_spectral')
        ax.set_title(title)
        ax.axis('off')

        props = measure.regionprops(mask)
        for prop in props:
            y, x = prop.centroid  # (row, column) format
            label_number = prop.label
            ax.text(x, y, str(label_number), color='white', fontsize=8, ha='center', va='center')

    plt.tight_layout()
    relabel_mask_path = os.path.join(mask_dir, f"{basename}_relabel_mask.png")
    io.imsave(relabel_mask_path, new_nucleus_labels.astype(np.uint16))
    plt.show()
    
    
    ##################################################################
    #step8:
    # Load the fluorescence image (ensure it's in grayscale or RGB format)
    image = io.imread(path)
    print(f"Image shape: {image.shape}")
    channel1 = image[:, :, 1]
    fluorescence_img = channel1
    #cytoplasm_mask  = io.imread('/Volumes/lsa-kkoutmou/migratedData/Private_Folders/mlruan/Data/EYFP_20241004_red_dye/20250324-IF-DCF/plate1_IF/NES-new/select/bright-nes2_cp_masks.png')
    #nucleus_mask  = io.imread("/Volumes/lsa-kkoutmou/migratedData/Private_Folders/mlruan/Data/EYFP_20241004_red_dye/20250324-IF-DCF/plate1_IF/NES-new/select/blue-nes_cp_masks.png")
    
    # Find boundaries of the nucleus mask
    nucleus_boundaries = find_boundaries(nucleus_mask, mode='outer', background=0)
    
    # Find boundaries of the cytoplasm mask
    cytoplasm_boundaries = find_boundaries(cytoplasm_mask, mode='outer', background=0)
    
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Display the fluorescence image
    ax.imshow(fluorescence_img, cmap='gray')
    
    # Overlay the nucleus boundaries in red
    ax.contour(nucleus_boundaries, colors='r', linewidths=1)
    
    # Overlay the cytoplasm boundaries in blue
    ax.contour(cytoplasm_boundaries, colors='b', linewidths=1)
    
    # Set axis labels and title
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    ax.set_title('Fluorescence Image with Nucleus and Cytoplasm Boundaries')
    
    # Hide axes for better visualization
    ax.axis('off')
    
    # Show the plot
    plt.tight_layout()
    plot_path = os.path.join(mask_dir, f"{basename}_boundaries.png")
    plt.savefig(plot_path, dpi=300)
    plt.show()
    plt.close()
    
    ######################################################
    #step6: generate the csv files
    image = io.imread(path)
    print(f"Image shape: {image.shape}")
    channel1 = image[:, :, 1]
    
    # Save channel 1 as JPG
    channel1_8bit = img_as_ubyte(channel1 / channel1.max()) if channel1.max() > 0 else channel1
    jpg_path = os.path.splitext(path)[0] + "_channel1.jpg"
    #io.imsave(jpg_path, channel1_8bit)
    #rint(f"Channel 1 saved as: {jpg_path}")
    
    # Show original fluorescence + masks
    if 'cytoplasm_mask' in locals() and 'new_nucleus_labels' in locals():
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        masks = [cytoplasm_mask, new_nucleus_labels]
        titles = ['Raw Fluorescence (Channel 1)', 'Cytoplasm Mask Overlay', 'Nucleus Mask Overlay']
    
        # Show raw fluorescence
        axes[0].imshow(channel1, cmap='gray')
        axes[0].set_title(titles[0])
        axes[0].axis('off')
    
        # Show overlays
        for ax, mask, title in zip(axes[1:], masks, titles[1:]):
            ax.imshow(channel1, cmap='gray')  # Fluorescence as base
            ax.set_title(title)
            ax.axis('off')
    
            # Add contours and labels
            props = regionprops(mask)
            for prop in props:
                y, x = prop.centroid
                label_number = prop.label
                ax.text(x, y, str(label_number), color='white', fontsize=8, ha='center', va='center')
    
            contours = measure.find_contours(mask, level=0.5)
            for contour in contours:
                ax.plot(contour[:, 1], contour[:, 0], linewidth=1, color='cyan')
    
        plt.tight_layout()
        overlay_path = os.path.join(mask_dir, f"{basename}_overlays.png")
        plt.savefig(overlay_path, dpi=300)
        plt.show()
        plt.close()
    else:
        print("cytoplasm_mask or new_nucleus_labels not defined.")
        
    if 'cytoplasm_mask' in locals():
        # Measure mean intensity and area per region in the cytoplasm mask
        cytoplasm_props = regionprops_table(
            cytoplasm_mask,
            intensity_image=channel1,
            properties=('label', 'mean_intensity', 'area')
        )
    
        # Convert to DataFrame
        cytoplasm_df = pd.DataFrame(cytoplasm_props)
    
        # Show the result
        print(cytoplasm_df)
    else:
        done
    
    # Measure properties for nucleus regions
    nucleus_props = regionprops_table(
        new_nucleus_labels,
        intensity_image=channel1,
        properties=('label', 'mean_intensity','area')
    )
    
    # Convert to DataFrame for easier analysis
    nucleus_df = pd.DataFrame(nucleus_props)
    
    # Display the DataFrame
    print(nucleus_df)

   ######################################################
    #step7: calculate ratio of nucleus vs whole
    nucleus_df['total_intensity'] = nucleus_df['area'] * nucleus_df['mean_intensity']
    cytoplasm_df['total_intensity'] = cytoplasm_df['area'] * cytoplasm_df['mean_intensity'] 
    
    # Merge nucleus and cytoplasm DataFrames on 'label'
    merged_df = pd.merge(
        nucleus_df[['label', 'area', 'mean_intensity', 'total_intensity']],
        cytoplasm_df[['label', 'area', 'mean_intensity', 'total_intensity']],
        on='label',
        suffixes=('_nucleus', '_cytoplasm')
    )
    
    # Calculate cytoplasmic-only total intensity and the intensity ratio
    merged_df['total_intensity_cytoplasm2'] = merged_df['total_intensity_cytoplasm'] - merged_df['total_intensity_nucleus']
    merged_df['intensity_ratio'] = merged_df['total_intensity_nucleus'] / merged_df['total_intensity_cytoplasm']
    
    # Save to CSV
    print(merged_df['intensity_ratio'].mean())
    csv_path = os.path.join(csv_dir, f"{basename}_intensity_ratios.csv")
    merged_df.to_csv(csv_path, index=False)

    ######################################################
    #step7: plot
    sns.set(style="whitegrid", font_scale=1.4)
    
    plt.figure(figsize=(8, 6))
    
    # Plot histogram with KDE overlay
    sns.histplot(merged_df['intensity_ratio'], 
                 bins=30, 
                 color='skyblue', 
                 edgecolor='black', 
                 kde=True)
    
    # Compute mean
    mean_val = merged_df['intensity_ratio'].mean()
    
    # Add vertical line for mean
    plt.axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean = {mean_val:.2f}')
    
    # Add label annotation
    plt.text(mean_val + 0.02, plt.ylim()[1]*0.9, f'{mean_val:.2f}', 
             color='red', fontsize=12, weight='bold')
    
    # Titles and labels
    plt.title('Distribution of Nucleus-to-Cytoplasm Total Intensity Ratio', fontsize=16, weight='bold')
    plt.xlabel('Intensity Ratio (Nucleus / Cytoplasm)', fontsize=14)
    plt.ylabel('Cell Count', fontsize=14)
    
    # Adjust the appearance
    plt.grid(True, linestyle='--', linewidth=0.5)
    sns.despine(trim=True)
    plt.legend()
    plt.tight_layout()
    
    # Save plot
    hist_path = os.path.join(plot_dir, f"{basename}_ratio_histogram.png")
    plt.savefig(hist_path, dpi=300)
    plt.show()
    plt.close()

    return new_nucleus_labels


In [ ]:
#all samples
input_dir = "/Volumes/lsa-kkoutmou/migratedData/Private_Folders/mlruan/First_paper/data_R_script/Fig3_imaging_IF/imaging_quantification_all_pics_excels/cellpose/data"
tif_files = glob.glob(os.path.join(input_dir, "**", "*.tif"), recursive=True)
tif_files 

In [ ]:
# Loop through all .tif files and apply your function
for tif_path in tif_files:
    print(f"Processing {tif_path}")
    try:
        result = load_and_segment_image(tif_path,  7, 1,0,20)
    except Exception as e:
        print(f"Error processing {tif_path}: {e}")